In [29]:
import pandas as pd
import numpy as np
from collections import defaultdict

# [Painel  COVID RS](https://ti.saude.rs.gov.br/covid19/?fbclid=IwAR3fNK62crpaeD1AmlPfz6_lNB7ATDLcW5YaQEmF2bMUD-pyUvw2--qhaEA)

## Infos iniciais

In [30]:
ses_rs = pd.read_csv("dataset/ses_rs.csv", sep=";")

In [31]:
ses_rs.head()

,COD_IBGE,MUNICIPIO,COD_REGIAO_COVID,REGIAO_COVID,SEXO,FAIXAETARIA,CRITERIO,DATA_CONFIRMACAO,DATA_SINTOMAS,DATA_EVOLUCAO,...,DATA_INCLUSAO_OBITO,DATA_EVOLUCAO_ESTIMADA,RACA_COR,ETNIA_INDIGENA,PROFISSIONAL_SAUDE,BAIRRO,HOSPITALIZACAO_SRAG,FONTE_INFORMACAO,PAIS_NASCIMENTO,PES_PRIV_LIBERDADE
0,430003,ACEGUÁ,16,BAGE - R22,Feminino,20 a 29,RT-PCR,11/09/2020,08/09/2020,NaN,...,NaN,NaN,NAO INFORMADO,NaN,NAO INFORMADO,NaN,NAO,E-SUS,NaN,NAO
1,430005,ÁGUA SANTA,14,PASSO FUNDO - R17 R18 R19,Feminino,30 a 39,TESTE RÁPIDO,03/08/2020,25/07/2020,NaN,...,NaN,08/08/2020,BRANCA,NAO ENCONTRADO,NAO,INTERIOR,NAO,E-SUS,BRASIL,NAO
2,430005,ÁGUA SANTA,14,PASSO FUNDO - R17 R18 R19,Feminino,20 a 29,TESTE RÁPIDO,26/06/2020,26/06/2020,NaN,...,NaN,10/07/2020,BRANCA,NAO ENCONTRADO,NAO,CENTRO,NAO,E-SUS,BRASIL,NAO
3,430005,ÁGUA SANTA,14,PASSO FUNDO - R17 R18 R19,Masculino,20 a 29,TESTE RÁPIDO,26/06/2020,25/06/2020,NaN,...,NaN,09/07/2020,BRANCA,NAO ENCONTRADO,NAO,CENTRO,NAO,E-SUS,BRASIL,NAO
4,430005,ÁGUA SANTA,14,PASSO FUNDO - R17 R18 R19,Feminino,40 a 49,TESTE RÁPIDO,07/08/2020,29/07/2020,NaN,...,NaN,12/08/2020,BRANCA,NAO ENCONTRADO,NAO,CENTRO,NAO,E-SUS,BRASIL,NAO


In [32]:
ses_rs.columns

Index(['COD_IBGE', 'MUNICIPIO', 'COD_REGIAO_COVID', 'REGIAO_COVID', 'SEXO',
       'FAIXAETARIA', 'CRITERIO', 'DATA_CONFIRMACAO', 'DATA_SINTOMAS',
       'DATA_EVOLUCAO', 'EVOLUCAO', 'HOSPITALIZADO', 'FEBRE', 'TOSSE',
       'GARGANTA', 'DISPNEIA', 'OUTROS', 'CONDICOES', 'GESTANTE',
       'DATA_INCLUSAO_OBITO', 'DATA_EVOLUCAO_ESTIMADA', 'RACA_COR',
       'ETNIA_INDIGENA', 'PROFISSIONAL_SAUDE', 'BAIRRO', 'HOSPITALIZACAO_SRAG',
       'FONTE_INFORMACAO', 'PAIS_NASCIMENTO', 'PES_PRIV_LIBERDADE'],
      dtype='object')

## Limpeza

### Drop de colunas identificadoras

In [33]:
ses_rs = ses_rs[['SEXO', 'FAIXAETARIA', 'EVOLUCAO', 'HOSPITALIZADO', 'FEBRE', 'TOSSE',
       'GARGANTA', 'DISPNEIA', 'CONDICOES', 'GESTANTE', 'RACA_COR',
       'ETNIA_INDIGENA']]

### Drop de linha com dados faltantes

In [34]:
ses_rs.dropna(subset=['SEXO', 'FAIXAETARIA', 'EVOLUCAO', 'HOSPITALIZADO', 'FEBRE', 'TOSSE',
       'GARGANTA', 'DISPNEIA', 'GESTANTE', 'RACA_COR','ETNIA_INDIGENA'], inplace=True)

### Análise dos valores possíveis para cada variável

In [35]:
for column in ses_rs.columns:
    print(column+"\n")
    print(ses_rs[column].value_counts(normalize=True))
    print("-"*40)

SEXO

Feminino     0.502818
Masculino    0.497182
Name: SEXO, dtype: float64
----------------------------------------
FAIXAETARIA

30 a 39      0.231215
40 a 49      0.189480
20 a 29      0.181659
50 a 59      0.152381
60 a 69      0.089506
70 a 79      0.043627
15 a 19      0.035183
80 e mais    0.025314
10 a 14      0.017547
05 a 09      0.016421
01 a 04      0.014259
<1           0.003408
Name: FAIXAETARIA, dtype: float64
----------------------------------------
EVOLUCAO

RECUPERADO           0.924897
EM ACOMPANHAMENTO    0.054383
OBITO                0.020721
Name: EVOLUCAO, dtype: float64
----------------------------------------
HOSPITALIZADO

NAO    0.927088
SIM    0.072912
Name: HOSPITALIZADO, dtype: float64
----------------------------------------
FEBRE

NAO    0.668248
SIM    0.331752
Name: FEBRE, dtype: float64
----------------------------------------
TOSSE

NAO    0.566369
SIM    0.433631
Name: TOSSE, dtype: float64
----------------------------------------
GARGANTA

NAO    0

In [36]:
ses_rs.CONDICOES

1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
         ... 
175344    NaN
175345    NaN
175346    NaN
175347    NaN
175348    NaN
Name: CONDICOES, Length: 166983, dtype: object

#### Encoding das colunas categóricas nominais

In [37]:
# ses_rs.replace({"Feminino": 1, "Masculino": 0, "SIM": 1, "NAO": 0}, inplace=True)
# ses_rs[['SEXO','HOSPITALIZADO', 'FEBRE', 'TOSSE','GARGANTA', 'DISPNEIA',  'GESTANTE']].astype(np.int8)

#### Drop de ETNIA_INDIGENA e RACA_COR. Coluna ETNIA_INDIGENA está com muitos dados faltantes e coluna RACA_COR está muito enviesada

In [38]:
ses_rs.drop(columns=["ETNIA_INDIGENA", 'RACA_COR'], inplace=True)

#### Encoding da coluna "EVOLUCAO". Essa coluna e a coluna "HOSPITALIZADO" serão as variáveis dependentes

In [39]:
# ses_rs.EVOLUCAO = ses_rs.EVOLUCAO.replace({"EM ACOMPANHAMENTO": 0, "RECUPERADO": 0, "OBITO": 1})
# ses_rs.rename(columns={"EVOLUCAO": "obito", "HOSPITALIZADO": "hospitalizado"}, inplace=True)
# ses_rs

#### Encodig da faixa etária

In [40]:
# ses_rs.FAIXAETARIA.replace({'<1': 0.5,
#                             '01 a 04': 2.5, 
#                             '05 a 09': 6.5, 
#                             '10 a 14': 12,
#                             '15 a 19': 17, 
#                             '20 a 29': 24.5, 
#                             '30 a 39': 34.5,
#                             '40 a 49': 44.5, 
#                             '50 a 59': 54.5, 
#                             '60 a 69': 64.5, 
#                             '70 a 79': 74.5, 
#                             '80 e mais': 88, 
#                             }, inplace=True)

In [41]:
ses_rs.reset_index(drop=True, inplace=True)

### Limpeza da coluna 'CONDICOES'

In [42]:
ses_rs.CONDICOES = ses_rs.CONDICOES.apply(lambda condicoes:
                      condicoes.split(",") if condicoes is not np.nan else [])

condicoes_existentes = np.unique(np.concatenate(ses_rs["CONDICOES"].values))

condicoes_existentes

array([' 4 ou 5)', ' Asm', ' Asma', ' Diabetes', ' Diabetes mellitu',
       ' Diabetes mellitus', ' Doença Cardiovascular Crônica',
       ' Doença Hematológica Crônic', ' Doença Hematológica Crônica',
       ' Doença Hepática Crônic', ' Doença Hepática Crônica',
       ' Doença Neurológica Crônic', ' Doença Neurológica Crônica',
       ' Doença Renal Crônic', ' Doença Renal Crônica',
       ' Doenças cardíacas crônicas',
       ' Doenças renais crônicas em estágio avançado (graus 3',
       ' Doenças respiratórias crônicas descompensadas', ' Gestante',
       ' Gestante de alto risco', ' Imunodeficiênci', ' Imunodeficiência',
       ' Imunossupressão', ' Obesidad', ' Obesidade',
       ' Outra Pneumatopatia Crônic', ' Outra Pneumatopatia Crônica',
       ' Outro', ' Outros',
       ' Portador de doenças cromossômicas ou estado de fragilidade imunológica',
       ' Síndrome de Down', 'Asma', 'Diabetes', 'Diabetes mellitus',
       'Doença Cardiovascular Crônica', 'Doença Hematológica 

In [43]:
dict_correcao_ort = {"asma": [" Asma", 
                              "Asm", 
                              "Asma"],
                    "diabetes": [' Diabetes', 
                                 ' Diabetes mellitu', 
                                 ' Diabetes mellitus', 
                                 "Diabetes", 
                                 'Diabetes mellitus'],
                    "doenca_cardiovascular": [' Doença Cardiovascular Crônica', 
                                              ' Doenças cardíacas crônicas', 
                                              ' Doenças cardíacas crônicas', 
                                              'Doença Cardiovascular Crônica',
                                              'Doenças cardíacas crônicas'],
                    "doenca_hematologica": [' Doença Hematológica Crônic', 
                                            ' Doença Hematológica Crônica', 
                                            'Doença Hematológica Crônic',
                                            'Doença Hematológica Crônica'],
                    "doenca_hepatica": [' Doença Hepática Crônic', 
                                        ' Doença Hepática Crônica', 
                                        'Doença Hepática Crônica'],
                    "doenca_neurologica": [' Doença Neurológica Crônic', 
                                           ' Doença Neurológica Crônica'],
                    "doenca_renal": [" Doença Renal Crônic", 
                                     " Doença Renal Crônica", 
                                     ' Doenças renais crônicas em estágio avançado (graus 3',' 4 ou 5)',
                                     'Doença Renal Crônica'],
                    "gestante": [' Gestante',  
                                 'Gestante'],
                    "gestante_alto_risco": [' Gestante de alto risco', 
                                            'Gestante de alto risco'],
                    "imunodeficiencia": [' Imunodeficiênci', 
                                         ' Imunodeficiência', 
                                         ' Imunossupressão',
                                         ' Portador de doenças cromossômicas ou estado de fragilidade imunológica',
                                         'Imunodeficiência', 
                                         'Imunossupressão'],
                    "obesidade": [' Obesidad', 
                                  ' Obesidade', 
                                  'Obesidade', ],
                    "pneumopatia": [' Doenças respiratórias crônicas descompensadas',
                                    ' Outra Pneumatopatia Crônic', 
                                    ' Outra Pneumatopatia Crônica', 
                                    "Pneumatopatia Crônica",
                                    'Doenças respiratórias crônicas descompensadas', 
                                    'Outra Pneumatopatia Crônica',
                                    'Pneumatopatia Crônica', 
                                    'Portador de doenças cromossômicas ou estado de fragilidade imunológica'],
                    "doenca_neurologica": ['Doença Neurológica Crônica'],
                    "puerpera": ['Puérpera', 
                                 'Puérpera (até 45 dias do parto)'],
                    "sindrome_de_down": [' Síndrome de Down',  
                                         'Síndrome de Down']     
                    }

dict_correcao = defaultdict(lambda:"Outros")
for chave, valores in dict_correcao_ort.items():
        for valor in valores:
            dict_correcao[valor] = chave
dict_correcao

defaultdict(<function __main__.<lambda>()>,
            {' Asma': 'asma',
             'Asm': 'asma',
             'Asma': 'asma',
             ' Diabetes': 'diabetes',
             ' Diabetes mellitu': 'diabetes',
             ' Diabetes mellitus': 'diabetes',
             'Diabetes': 'diabetes',
             'Diabetes mellitus': 'diabetes',
             ' Doença Cardiovascular Crônica': 'doenca_cardiovascular',
             ' Doenças cardíacas crônicas': 'doenca_cardiovascular',
             'Doença Cardiovascular Crônica': 'doenca_cardiovascular',
             'Doenças cardíacas crônicas': 'doenca_cardiovascular',
             ' Doença Hematológica Crônic': 'doenca_hematologica',
             ' Doença Hematológica Crônica': 'doenca_hematologica',
             'Doença Hematológica Crônic': 'doenca_hematologica',
             'Doença Hematológica Crônica': 'doenca_hematologica',
             ' Doença Hepática Crônic': 'doenca_hepatica',
             ' Doença Hepática Crônica': 'doenca

In [44]:
ses_rs.CONDICOES = ses_rs.CONDICOES.apply(lambda condicoes_paciente: [dict_correcao[condicao] for condicao in condicoes_paciente])
condicoes_existentes = set(dict_correcao.values())
matrix_condicoes = ses_rs.CONDICOES.apply(lambda condicoes_paciente:
                                                      [int(condicao in condicoes_paciente) for condicao in condicoes_existentes])

ses_rs = pd.concat([ses_rs, pd.DataFrame(data=matrix_condicoes.tolist(), columns=condicoes_existentes)], axis=1)
ses_rs.head()

,SEXO,FAIXAETARIA,EVOLUCAO,HOSPITALIZADO,FEBRE,TOSSE,GARGANTA,DISPNEIA,CONDICOES,GESTANTE,...,asma,sindrome_de_down,Outros,diabetes,imunodeficiencia,doenca_cardiovascular,doenca_hematologica,doenca_renal,pneumopatia,puerpera
0,Feminino,30 a 39,RECUPERADO,NAO,NAO,SIM,NAO,NAO,[],NAO,...,0,0,0,0,0,0,0,0,0,0
1,Feminino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,[],NAO,...,0,0,0,0,0,0,0,0,0,0
2,Masculino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,[],NAO,...,0,0,0,0,0,0,0,0,0,0
3,Feminino,40 a 49,RECUPERADO,NAO,NAO,NAO,NAO,NAO,[],NAO,...,0,0,0,0,0,0,0,0,0,0
4,Feminino,30 a 39,RECUPERADO,NAO,NAO,NAO,NAO,NAO,[],NAO,...,0,0,0,0,0,0,0,0,0,0


In [45]:
ses_rs.drop(columns=["CONDICOES", "Outros"], inplace=True)

In [46]:
ses_rs.columns

Index(['SEXO', 'FAIXAETARIA', 'EVOLUCAO', 'HOSPITALIZADO', 'FEBRE', 'TOSSE',
       'GARGANTA', 'DISPNEIA', 'GESTANTE', 'gestante', 'gestante_alto_risco',
       'obesidade', 'doenca_neurologica', 'doenca_hepatica', 'asma',
       'sindrome_de_down', 'diabetes', 'imunodeficiencia',
       'doenca_cardiovascular', 'doenca_hematologica', 'doenca_renal',
       'pneumopatia', 'puerpera'],
      dtype='object')

### A interseção das colunas 'gestante_alto_risco' e 'gestante' igual a coluna 'GESTANTE'. 

In [47]:
all(ses_rs.apply(lambda row: (row["GESTANTE"] == row["gestante"]) or (row["GESTANTE"] == row['gestante_alto_risco']), axis=1))

False

In [48]:
ses_rs.drop(columns=["gestante_alto_risco", "gestante"], inplace=True)

## Resultado final

In [49]:
ses_rs.head()

,SEXO,FAIXAETARIA,EVOLUCAO,HOSPITALIZADO,FEBRE,TOSSE,GARGANTA,DISPNEIA,GESTANTE,obesidade,...,doenca_hepatica,asma,sindrome_de_down,diabetes,imunodeficiencia,doenca_cardiovascular,doenca_hematologica,doenca_renal,pneumopatia,puerpera
0,Feminino,30 a 39,RECUPERADO,NAO,NAO,SIM,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
1,Feminino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
2,Masculino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
3,Feminino,40 a 49,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
4,Feminino,30 a 39,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
ses_rs.columns

Index(['SEXO', 'FAIXAETARIA', 'EVOLUCAO', 'HOSPITALIZADO', 'FEBRE', 'TOSSE',
       'GARGANTA', 'DISPNEIA', 'GESTANTE', 'obesidade', 'doenca_neurologica',
       'doenca_hepatica', 'asma', 'sindrome_de_down', 'diabetes',
       'imunodeficiencia', 'doenca_cardiovascular', 'doenca_hematologica',
       'doenca_renal', 'pneumopatia', 'puerpera'],
      dtype='object')

In [51]:
ses_rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166983 entries, 0 to 166982
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   SEXO                   166983 non-null  object
 1   FAIXAETARIA            166983 non-null  object
 2   EVOLUCAO               166983 non-null  object
 3   HOSPITALIZADO          166983 non-null  object
 4   FEBRE                  166983 non-null  object
 5   TOSSE                  166983 non-null  object
 6   GARGANTA               166983 non-null  object
 7   DISPNEIA               166983 non-null  object
 8   GESTANTE               166983 non-null  object
 9   obesidade              166983 non-null  int64 
 10  doenca_neurologica     166983 non-null  int64 
 11  doenca_hepatica        166983 non-null  int64 
 12  asma                   166983 non-null  int64 
 13  sindrome_de_down       166983 non-null  int64 
 14  diabetes               166983 non-null  int64 
 15  

In [52]:
ses_rs

,SEXO,FAIXAETARIA,EVOLUCAO,HOSPITALIZADO,FEBRE,TOSSE,GARGANTA,DISPNEIA,GESTANTE,obesidade,...,doenca_hepatica,asma,sindrome_de_down,diabetes,imunodeficiencia,doenca_cardiovascular,doenca_hematologica,doenca_renal,pneumopatia,puerpera
0,Feminino,30 a 39,RECUPERADO,NAO,NAO,SIM,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
1,Feminino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
2,Masculino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
3,Feminino,40 a 49,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
4,Feminino,30 a 39,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166978,Feminino,80 e mais,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
166979,Masculino,20 a 29,RECUPERADO,NAO,NAO,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
166980,Masculino,50 a 59,RECUPERADO,NAO,SIM,NAO,NAO,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0
166981,Masculino,50 a 59,RECUPERADO,NAO,SIM,NAO,SIM,NAO,NAO,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
for column in ses_rs.columns:
    print(column+"\n")
    print(ses_rs[column].value_counts(normalize=True))
    print("-"*40)

SEXO

Feminino     0.502818
Masculino    0.497182
Name: SEXO, dtype: float64
----------------------------------------
FAIXAETARIA

30 a 39      0.231215
40 a 49      0.189480
20 a 29      0.181659
50 a 59      0.152381
60 a 69      0.089506
70 a 79      0.043627
15 a 19      0.035183
80 e mais    0.025314
10 a 14      0.017547
05 a 09      0.016421
01 a 04      0.014259
<1           0.003408
Name: FAIXAETARIA, dtype: float64
----------------------------------------
EVOLUCAO

RECUPERADO           0.924897
EM ACOMPANHAMENTO    0.054383
OBITO                0.020721
Name: EVOLUCAO, dtype: float64
----------------------------------------
HOSPITALIZADO

NAO    0.927088
SIM    0.072912
Name: HOSPITALIZADO, dtype: float64
----------------------------------------
FEBRE

NAO    0.668248
SIM    0.331752
Name: FEBRE, dtype: float64
----------------------------------------
TOSSE

NAO    0.566369
SIM    0.433631
Name: TOSSE, dtype: float64
----------------------------------------
GARGANTA

NAO    0

In [54]:
ses_rs.to_csv("ses_rs_limpo.csv")